In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import pandas_datareader as pdr

# API key for Tiingo
key = "30c0973b8a648106ae38faea9031b9a3924c7469"

# Getting the Disney (DIS) dataset
df_dis = pdr.get_data_tiingo('DIS', api_key=key)

# Save the dataset to a CSV file
df_dis.to_csv('DIS.csv')

# Read the data and parse the date column
df_dis = pd.read_csv('DIS.csv', parse_dates=['date'], index_col='date')

# Split the data into training and test sets
df_training_dis = df_dis[df_dis.index <= '2022-01-01'].copy()
df_test_dis = df_dis[df_dis.index > '2022-01-01'].copy()

# Extract the 'adjClose' values for training and test
training_data_dis = df_training_dis['close']

# Decompose the training data into trend, seasonality, and residuals using STL
stl = STL(training_data_dis, seasonal=13)  # Adjust the seasonal period as needed
stl_result = stl.fit()

# Train an ARIMA model on the residuals
residuals = stl_result.resid
arima_model = ARIMA(residuals, order=(5,1,0))  # You can tune the ARIMA hyperparameters
arima_fit = arima_model.fit()

# Forecast residuals for the test period
residual_forecast = arima_fit.forecast(steps=len(df_test_dis))

# Add the seasonal and trend components back to obtain the full forecast
forecasted_values = stl_result.trend + stl_result.seasonal + pd.Series(residual_forecast, index=df_test_dis.index)

# Evaluate the model (MSE)
mse = mean_squared_error(df_test_dis['close'], forecasted_values)
print(f"Mean Squared Error (MSE): {mse}")

# Visualize actual vs. predicted stock prices
plt.figure(figsize=(14, 7))
plt.plot(df_test_dis.index, df_test_dis['adjClose'], color='blue', label='Actual Stock Price')
plt.plot(df_test_dis.index, forecasted_values, color='red', label='Predicted Stock Price')
plt.title('Disney Stock Price Prediction (STL with ARIMA)')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()


c:\Users\sneha\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:177: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ImportError: DLL load failed while importing lapack_lite: The specified module could not be found.